# Домашнее задание №1: линейная регрессия и векторное дифференцирование (10 баллов).

* Максимальное количество баллов за задания в ноутбуке - 11, но больше 10 оценка не ставится, поэтому для получения максимальной оценки можно сделать не все задания.

* Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1.

In [141]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [142]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [143]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.892056207699224e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [144]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

3.870036668388616e-12


array([ 1.74221905e-09, -4.17724606e-08, -1.79051871e-08, -8.79903529e-09,
       -3.17528632e-10, -6.68193091e-10, -7.28235946e-09, -1.08742286e-08,
        2.16586724e-10,  9.18615841e-08, -1.78301001e-09,  3.91326818e+01,
       -5.07349337e-08,  1.05284678e-08, -1.56047650e-08, -7.09338778e-08,
       -6.31623867e-09, -1.75350413e-09,  5.36243315e-09,  2.21020084e-11,
        3.33239287e-08, -2.97911061e-08,  3.56629148e-08, -6.11159535e-08,
       -5.31343092e-10,  5.46858996e+01,  5.39089391e-08,  1.91044452e-08,
        6.83750874e-08,  9.00626857e+01,  2.82383751e-08, -8.88254933e-09,
        8.20042253e-10,  2.91540252e-08, -6.05131384e-08, -1.94689944e-08,
       -2.45091921e-08,  6.96162624e-08,  1.33923693e-08,  3.41558054e+01,
        5.36032341e-08,  6.45387341e-08,  1.73950002e-08,  8.60891719e-08,
        5.56214324e-09, -2.21727122e-08,  7.85650818e-08, -2.38166763e-08,
        4.30995304e+01, -9.24404100e-09, -1.26308883e-08,  9.07501988e-08,
       -4.45844925e-09,  

***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

In [145]:
#Различие в метриках обусловлено выбором параметров в методе градиентного спуска (они отличаются от тех, которые заданы в изначальной функции LinearRegression).
#Alpha слишком сильно наказывает за веса (возможно, они слишком большие)

In [173]:
reg = SGDRegressor(alpha=0).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

5.1739612732157225e-25


In [104]:
#Вот и получилось через уменьшение параметра alpha сделать MSE с точностью до 25 знака таким же, как MSE у линейной регрессии

## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс. 

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и второй варианты) или l2 (третий и четвертый варианты) регуляризацию. 

In [178]:
class LinearRegression(object):
    def __init__(self, alpha=0.5, l_ratio=0.000001, tol=0.001, max_iter=1000):
        '''
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - параметр регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        '''
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter
             
    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        '''
        iteracii = 0
        Xn = np.hstack([X, np.ones([X.shape[0], 1])])
        w_0 = np.random.uniform(-2, 2, (Xn.shape[1])) #инициализирую вектор весов (рандомный)
         
        #так как я не знаю, идёт ли в исходной матрице первый столбец единичным, сам его добавляю
        Xt = Xn.transpose()
        
        mse_0 = (1 / Xn.shape[0]) * np.linalg.norm((Xn @ w_0 - y))**2 + self.l_ratio * (np.linalg.norm(w_0[:-1])**2)
        #считаю MSE для этих весов
        crit = 100000
        
        while iteracii <= self.max_iter and crit >= self.tol: #условие на макс. кол-во итераций и критерий останова
            grad_w = (2 / Xn.shape[0]) * Xt @ ((Xn @ w_0 - y)) + 2 * self.l_ratio * w_0
             
            w_1 = w_0 - self.alpha * grad_w
            
            mse_1 = (1 / Xn.shape[0]) * np.linalg.norm((Xn @ w_1 - y))**2 + self.l_ratio * (np.linalg.norm(w_1[:-1])**2)
            crit = abs(mse_1 - mse_0)
            w_0 = w_1
            mse_0 = mse_1
            iteracii += 1
            
        self.w_0 = w_0

   
    def predict(self, X):
        '''
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        '''
        Xn = np.hstack([X, np.ones([X.shape[0], 1])])
        return Xn.dot(self.w_0)

In [179]:
my_reg = LinearRegression()
my_reg.fit(X, y)
print(mean_squared_error(y, my_reg.predict(X)))
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

3.6451703761882005e-06
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso, **первый и второй вариант**) или с l2-регуляризацией (from sklearn.linear_model import Ridge, **третий и четвертый вариант**)
* со значением параметра регуляризации **0.1 - для первого и третьего варианта, 0.01 - для второго и четвертого варианта**. 

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [180]:
from sklearn.linear_model import Ridge
Ridge_1 = Ridge(alpha=0.01).fit(X, y)
print(mean_squared_error(y, Ridge_1.predict(X)))

3.9446973557210053e-08


In [181]:
Ne_Ridge = LinearRegression(l_ratio=0.01)
Ne_Ridge.fit(X, y)
print(mean_squared_error(y, Ne_Ridge.predict(X)))

3.870030682487219


In [74]:
#Вывод: получается, что мой код не так хорошо подбирает параметры, MSE больше в 100000000 раз. Метод из коробки работает лучше. Я программист начального уровня

***Задание 6* (1 балл).***
Пусть $P, Q \in \mathbb{R}^{n\times n}$. Найдите $\nabla_Q tr(PQ)$

***Задание 7* (1 балл).***
Пусть $x, y \in \mathbb{R}^{n}, M \in \mathbb{R}^{n\times n}$. Найдите $\nabla_M x^T M y$

Решения заданий 6 и 7 можно написать на листочке и отправить в anytask вместе с заполненным ноутбуком.

In [159]:
SGDRegressor?

In [148]:
LinearRegression?